
---

## 🌐 CRUD Overview

CRUD stands for:

| Operation  | Description                     |
| ---------- | ------------------------------- |
| **Create** | Add new data to the database    |
| **Read**   | Retrieve data from the database |
| **Update** | Modify existing data            |
| **Delete** | Remove data                     |

We will implement all four using Flask + SQLAlchemy + SQLite.

---

## ✅ 1️⃣ Folder Structure

```
project/
│   app.py
└───templates/
      users.html
      add_user.html
      update_user.html
```

---

## ✅ 2️⃣ Setup Flask + SQLAlchemy

**app.py**

```python
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# Model
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50), nullable=False)
    email = db.Column(db.String(120), nullable=False)

@app.before_first_request
def create_tables():
    db.create_all()
```

---

## ✅ 3️⃣ Create Operation (Add User)

**Route in app.py**

```python
@app.route("/add", methods=["GET", "POST"])
def add_user():
    if request.method == "POST":
        name = request.form["name"]
        email = request.form["email"]
        new_user = User(name=name, email=email)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for("users"))
    return render_template("add_user.html")
```

**templates/add_user.html**

```html
<h1>Add User</h1>
<form method="POST">
    Name: <input type="text" name="name" required><br>
    Email: <input type="email" name="email" required><br>
    <input type="submit" value="Add User">
</form>
```

---

## ✅ 4️⃣ Read Operation (View Users)

**Route in app.py**

```python
@app.route("/users")
def users():
    users_list = User.query.all()
    return render_template("users.html", users=users_list)
```

**templates/users.html**

```html
<h1>All Users</h1>
<a href="{{ url_for('add_user') }}">Add New User</a>
<ul>
    {% for user in users %}
        <li>
            {{ user.name }} ({{ user.email }}) 
            <a href="{{ url_for('update_user', user_id=user.id) }}">Edit</a> | 
            <a href="{{ url_for('delete_user', user_id=user.id) }}">Delete</a>
        </li>
    {% endfor %}
</ul>
```

---

## ✅ 5️⃣ Update Operation

**Route in app.py**

```python
@app.route("/update/<int:user_id>", methods=["GET", "POST"])
def update_user(user_id):
    user = User.query.get_or_404(user_id)
    if request.method == "POST":
        user.name = request.form["name"]
        user.email = request.form["email"]
        db.session.commit()
        return redirect(url_for("users"))
    return render_template("update_user.html", user=user)
```

**templates/update_user.html**

```html
<h1>Update User</h1>
<form method="POST">
    Name: <input type="text" name="name" value="{{ user.name }}" required><br>
    Email: <input type="email" name="email" value="{{ user.email }}" required><br>
    <input type="submit" value="Update User">
</form>
```

---

## ✅ 6️⃣ Delete Operation

**Route in app.py**

```python
@app.route("/delete/<int:user_id>")
def delete_user(user_id):
    user = User.query.get_or_404(user_id)
    db.session.delete(user)
    db.session.commit()
    return redirect(url_for("users"))
```

---

## 🔎 How It Works

1. **Create** → `db.session.add()` + `db.session.commit()`
2. **Read** → `User.query.all()` or `User.query.get(id)`
3. **Update** → Modify object fields → `db.session.commit()`
4. **Delete** → `db.session.delete(obj)` → `db.session.commit()`

---

## ✅ Key Points

* `get_or_404(id)` → returns 404 if object not found
* Routes for **update/delete** include `user_id` as a URL parameter
* Combine with **forms** and **template rendering** for a full interactive app

---

